In [248]:
import pandas as pd
import numpy as np
from scipy.stats import logistic
import scipy.optimize
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

In [262]:
# Loading Data file with X and y in one large matrix
d = pd.read_csv('OriginalXy.csv')
(m,n) = d.shape
d = d.ix[:,1:n+1]
d.shape

(250, 101502)

In [263]:
colname = d.columns.tolist()
colname.remove('y')
X = d[colname]
y = d[['y']]
y = np.ravel(y)

In [265]:
# Chi-square significance assessment
chifit,pval  = chi2(X, y)
index = np.ravel(np.asarray(range(1,101502)))
chi1 = pd.DataFrame(chifit,index)
chi2 = pd.DataFrame(pval,index)
chir = pd.concat([chi1, chi2], axis=1)
chir.columns = ['chifit','pval']

# Select the columns/features with p value threshold of 0.01 or 0.05
sig1 = chir.index[chir.pval < 0.01]
sig1 = sig1.tolist()
sig2 = chir.index[chir.pval < 0.05]
sig2 = sig2.tolist()

In [322]:
# F-regression significance assessment
F,pval2 = f_regression(X,y)
F1 = pd.DataFrame(F,index)
F2 = pd.DataFrame(pval2,index)
F_fit = pd.concat([F1, F2], axis=1)
F_fit.columns = ['ffit','pval2']

fsig1 = F_fit.index[F_fit.pval2 < 0.01]
#sig1 = sig1.tolist()
fsig2 = F_fit.index[F_fit.pval2 < 0.05]
#sig2 = sig2.tolist()

colname3 = [colname[i] for i in fsig1]
X_n = X.ix[:,colname3]

colname2 = [colname[i] for i in sig2]
X_n = X.ix[:,colname2]

In [316]:
# Spliting data into training and testing sets
split = train_test_split(range(0,250),train_size = 0.8)[0]
opposplit = [x for x in range(0,250) if x not in split]
trainX = X_n.ix[split,:]
trainy = y[split]
testX = X_n.ix[opposplit,:]
testy = y[opposplit]

In [317]:
# L-1 Logistic Regression
lg = LogisticRegressionCV(cv=5,penalty='l1',solver='liblinear')
lfit = lg.fit(trainX,trainy)

In [318]:
# Get prediciton result
lfit = lg.fit(trainX,trainy)

# prediction by 0-1 class
ypred = lfit.predict(testX)
# prediction by probability
ypred2 = lfit.predict_proba(testX)

# Get 0-1 loss function (misclassification)
np.mean(ypred <> testy)

0.44

In [319]:
# Get prediction accuracy through built in function 
lfit.score(testX, testy)

0.56000000000000005